<a href="https://colab.research.google.com/github/ishika9/Stack-Overflow-tag-prediction/blob/main/Stack_overflow_label_prediction_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### IMPORTING LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import string
import nltk

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


### READING THE DATA

In [ ]:
question= pd.read_csv('Questions.csv', encoding='latin')
tags= pd.read_csv('Tags.csv', encoding='latin')

In [ ]:
question.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [ ]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [ ]:
print(question.shape,tags.shape)

(1264216, 7) (3750994, 2)


In [ ]:
print(question.Id.nunique(), tags.Id.nunique())

1264216 1264216


### MERGING THE DATA FRAMES

In [ ]:
tags['Tag']= tags['Tag'].astype(str)
grouped_tags = pd.DataFrame(tags.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags)))
grouped_tags.columns= ['Tag']
print(grouped_tags.head())
grouped_tags['Tag']= grouped_tags['Tag'].astype(str)
grouped_tags['Tag']= grouped_tags['Tag'].apply(lambda x: x.split())
# grouped_tags= grouped_tags.to_frame()
grouped_tags= grouped_tags.sort_values(by='Id')
print(grouped_tags.head())

                                                Tag
Id                                                 
80                          flex actionscript-3 air
90     svn tortoisesvn branch branching-and-merging
120                             sql asp.net sitemap
180  algorithm language-agnostic colors color-space
260         c# .net scripting compiler-construction
                                                   Tag
Id                                                    
80                         [flex, actionscript-3, air]
90   [svn, tortoisesvn, branch, branching-and-merging]
120                            [sql, asp.net, sitemap]
180  [algorithm, language-agnostic, colors, color-s...
260       [c#, .net, scripting, compiler-construction]


In [ ]:
grouped_tags1 = grouped_tags

In [ ]:
grouped_tags = grouped_tags1

In [ ]:
flat_list = [item for sublist in grouped_tags['Tag'].values for item in sublist]

keywords = nltk.FreqDist(flat_list)

keywords = nltk.FreqDist(keywords)

frequencies_words = keywords.most_common(100)
tags_features = [word[0] for word in frequencies_words]
tags_features

['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios',
 'mysql',
 'css',
 'sql',
 'asp.net',
 'objective-c',
 'ruby-on-rails',
 '.net',
 'c',
 'iphone',
 'angularjs']

In [ ]:
def most_common(tags):
    tags_filtered = []
    for i in range(0, len(tags)):
        if tags[i] in tags_features:
            tags_filtered.append(tags[i])
    return tags_filtered

In [ ]:
grouped_tags['Tag'] = grouped_tags['Tag'].apply(lambda x: most_common(x))
grouped_tags['Tag'] = grouped_tags['Tag'].apply(lambda x: x if len(x)>0 else None)

In [ ]:
grouped_tags.dropna(subset=['Tag'], inplace=True)

In [ ]:
print(grouped_tags.shape)

(850988, 2)


1. Merging Question and grouped_answer dataframes to get df
2. Merging df and grouped_answer dataframes to get df

In [ ]:
grouped_tags['Ids']= grouped_tags.index
question.columns= ['Ids', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body']
question= question.sort_values(by='Ids')
df= pd.merge(question,grouped_tags, on='Ids')

In [ ]:
df.head()

,Ids,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
1,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
2,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
3,470,71.0,2008-08-02T15:11:47Z,2016-03-26T05:23:29Z,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,[.net]
4,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,<p>I would like the version property of my app...,[c#]


In [ ]:
df.shape

(850988, 8)

### REMOVING UNNECESSARY VARIABLES

In [ ]:
df.drop(columns=['Ids', 'OwnerUserId', 'CreationDate', 'ClosedDate'],inplace=True)
df.head()

,Score,Title,Body,Tag
0,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
1,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
2,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
3,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,[.net]
4,79,Automatically update version number,<p>I would like the version property of my app...,[c#]


### FILTERING DATA BASED ON SCORE AND MOST FREQUENTLY USED TAGS

In [ ]:
print(df.Score.min(), df.Score.max())

-49 3613


In [ ]:
# df1= df1[df1['Tag_count']>=500]
df1= df[df['Score']>6]
df1.shape

(39714, 4)

For better prediction we will be using only those tags which have been repeated for atleast 500 times and the score is more than 5. Low scores mean that the question is either erroneous or does not have sufficient information.

In [ ]:
type(df1['Tag'])

pandas.core.series.Series

In [ ]:
df1['Tag']

0         [sql, asp.net]
1             [c#, .net]
2                  [c++]
3                 [.net]
4                   [c#]
               ...      
849386             [c++]
849833             [c++]
850049             [c++]
850201          [python]
850289          [python]
Name: Tag, Length: 39714, dtype: object

In [ ]:
# df1.Tag.value_counts().sort_values(ascending=False)

### CHECKING FOR MISSING VALUES

In [ ]:
print(df1.isnull().sum())

print('Shape of df1:',df1.shape)

Score    0
Title    0
Body     0
Tag      0
dtype: int64
Shape of df1: (39714, 4)


### CLEANING THE TEXT FOR TITLE AND BODY

1. Removing punctuation
2. Removing HTML tags (if required)
3. Changing text into lowercase
4. Splitting the text into words
5. Removing stopwords

#### PUNCTUATION & HTML TAGS REMOVAL, LOWERCASE, WORD TOKENIZATION

In [ ]:
import string
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text= text.replace(punctuation,'')
    return text

In [ ]:
df1['Title']= df1['Title'].astype(str)

df1['Title1']= df1['Title'].apply(remove_punctuation)
df1['Title1']=df1['Title1'].str.lower()
df1['Title1']= df1['Title1'].str.split()
df1['Title1'].head()

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\435767400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Title']= df1['Title'].astype(str)
C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\435767400.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Title1']= df1['Title'].apply(remove_punctuation)
C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\435767400.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

0                                 [aspnet, site, maps]
1    [adding, scripting, functionality, to, net, ap...
2    [should, i, use, nested, classes, in, this, case]
3          [homegrown, consumption, of, web, services]
4             [automatically, update, version, number]
Name: Title1, dtype: object

In [ ]:
df1['Body']= df1['Body'].astype(str)
import re

df1['Body1']= df1['Body'].apply(lambda x: re.sub('<[^<]+?>','',x))
df1['Body1'].head()

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\1142757768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Body']= df1['Body'].astype(str)
C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\1142757768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Body1']= df1['Body'].apply(lambda x: re.sub('<[^<]+?>','',x))


0    Has anyone got experience creating SQL-based A...
1    I have a little game written in C#. It uses a ...
2    I am working on a collection of classes used f...
3    I've been writing a few web services for a .ne...
4    I would like the version property of my applic...
Name: Body1, dtype: object

In [ ]:
df1['Body1']= df1['Body1'].apply(remove_punctuation)
df1['Body1']=df1['Body1'].str.lower()
df1['Body1']= df1['Body1'].str.split()
df1['Body1'].head()

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\2773665156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Body1']= df1['Body1'].apply(remove_punctuation)
C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\2773665156.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Body1']=df1['Body1'].str.lower()
C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\2773665156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

0    [has, anyone, got, experience, creating, sqlba...
1    [i, have, a, little, game, written, in, c, it,...
2    [i, am, working, on, a, collection, of, classe...
3    [ive, been, writing, a, few, web, services, fo...
4    [i, would, like, the, version, property, of, m...
Name: Body1, dtype: object

#### LEMMATIZATION

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lematizer= WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text=[lematizer.lemmatize(i) for i in text]
    return lem_text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Arihant\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Arihant\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df1['Title1']= df1['Title1'].apply(lambda x: word_lemmatizer(x))

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\1484448558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Title1']= df1['Title1'].apply(lambda x: word_lemmatizer(x))


In [ ]:
df1['Body1']= df1['Body1'].apply(lambda x: word_lemmatizer(x))

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\1860479539.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Body1']= df1['Body1'].apply(lambda x: word_lemmatizer(x))


#### STOPWORD REMOVAL

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Arihant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df1['Body1']=df1['Body1'].apply(lambda x: " ".join([w for w in x if w not in stopwords]))
df1['Body1'].head(5)

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\584209584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Body1']=df1['Body1'].apply(lambda x: " ".join([w for w in x if w not in stopwords]))


0    ha anyone got experience creating sqlbased asp...
1    little game written c us database backend trad...
2    working collection class used video playback r...
3    ive writing web service net app im ready consu...
4    would like version property application increm...
Name: Body1, dtype: object

In [ ]:
df1['Title1']=df1['Title1'].apply(lambda x: " ".join([w for w in x if w not in stopwords]))
df1['Title1'].head(5)

C:\Users\Arihant\AppData\Local\Temp\ipykernel_17152\1014082726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Title1']=df1['Title1'].apply(lambda x: " ".join([w for w in x if w not in stopwords]))


0                                   aspnet site map
1    adding scripting functionality net application
2                             use nested class case
3                 homegrown consumption web service
4               automatically update version number
Name: Title1, dtype: object

### FINAL DATAFRAME AFTER TEXT CLEANING

In [ ]:
df1= df1.drop(['Title', 'Body','Score'],axis=1)
df1.head()

,Tag,Title1,Body1
0,"[sql, asp.net]",aspnet site map,ha anyone got experience creating sqlbased asp...
1,"[c#, .net]",adding scripting functionality net application,little game written c us database backend trad...
2,[c++],use nested class case,working collection class used video playback r...
3,[.net],homegrown consumption web service,ive writing web service net app im ready consu...
4,[c#],automatically update version number,would like version property application increm...


### TF-IDF VECTORIZATION

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

df1['Title1']= df1['Title1'].astype(str)
vectorizer = TfidfVectorizer()
X1 = vectorizer.fit_transform(df1['Title1'].str.lower())

In [ ]:
df1['Body1']= df1['Body1'].astype(str)
vectorizer = TfidfVectorizer()
X2 = vectorizer.fit_transform(df1['Body1'].str.lower())

### CHANGING CATEGORICAL VARIABLES INTO NUMERIC

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le= LabelEncoder()
# df1['Tags']= le.fit_transform(df1['Tags'])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
multilabel= MultiLabelBinarizer()

In [ ]:
y= multilabel.fit_transform(df1['Tag'])

In [ ]:
df1['Tag']

0         [sql, asp.net]
1             [c#, .net]
2                  [c++]
3                 [.net]
4                   [c#]
               ...      
849386             [c++]
849833             [c++]
850049             [c++]
850201          [python]
850289          [python]
Name: Tag, Length: 39714, dtype: object

In [ ]:
y

array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

### SPLITTING THE DATASET INTO TRAIN AND TEST SET

In [ ]:
y.shape

(39714, 20)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X2, y, test_size=0.30, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(27799, 336071) (11915, 336071) (27799, 20) (11915, 20)


### APPLYING DIFFERENT ALGORITHMS

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
lr = LogisticRegression(C=20)

# Creating the model on Training Data

from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(lr)
model=clf.fit(x_train,y_train)
prediction=model.predict(x_test)
# Printing the Overall Accuracy of the model
from sklearn import metrics
print(metrics.multilabel_confusion_matrix(y_test, prediction))
F1_Score=metrics.classification_report(y_test, prediction).split()[-2]
print('Accuracy of the model:', F1_Score)

c:\Users\Arihant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Arihant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.or

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
sgd = SGDClassifier()
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(sgd)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.multilabel_confusion_matrix(y_test, y_pred))
F1_Score=metrics.classification_report(y_test, y_pred).split()[-2]
print('Accuracy of the model:', F1_Score)

[[[11197    14]
  [  661    43]]

 [[10627    20]
  [  418   850]]

 [[11687     3]
  [  135    90]]

 [[11585    19]
  [  258    53]]

 [[11468     7]
  [  407    33]]

 [[10096    75]
  [ 1329   415]]

 [[10843    49]
  [  606   417]]

 [[11411    30]
  [  212   262]]

 [[11395    15]
  [  454    51]]

 [[11225    71]
  [  404   215]]

 [[11489    12]
  [  363    51]]

 [[10182    61]
  [  852   820]]

 [[10445   136]
  [  735   599]]

 [[11212    47]
  [  377   279]]

 [[11584     7]
  [  186   138]]

 [[11483     6]
  [  380    46]]

 [[11161    24]
  [  420   310]]

 [[10812    21]
  [  410   672]]

 [[11503     6]
  [  178   228]]

 [[11545    31]
  [  235   104]]]
Accuracy of the model: 0.43


c:\Users\Arihant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.svm import LinearSVC
sv= LinearSVC()
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(sv)
model = clf.fit(x_train,y_train)
y_pred= model.predict(x_test)
F1_Score=metrics.classification_report(y_test, y_pred).split()[-2]
print('Accuracy of the model:', F1_Score)


Accuracy of the model: 0.55


c:\Users\Arihant\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Implementation of model

In [ ]:
x = [ 'how to write ml code in python and java i have data but do not know how to do it']
xt = vectorizer.transform(x)
clf.predict(xt)
multilabel.inverse_transform(clf.predict(xt))

[('python',)]

In [ ]:
x = ['which is better java, c++ or javascript.']
xt = vectorizer.transform(x)
clf.predict(xt)
multilabel.inverse_transform(clf.predict(xt))

[('java', 'javascript')]

In [ ]:
x = ['how to implement sql queries in python']
xt = vectorizer.transform(x)
clf.predict(xt)
multilabel.inverse_transform(clf.predict(xt))

[('python',)]